#LOL-predict

####Importing

In [2]:
import pandas as pd
import numpy as np

###### Getting dataset

In [3]:
matches = pd.read_csv('data/matches.csv')
participants = pd.read_csv('data/participants.csv')
stats1 = pd.read_csv('data/stats1.csv')
stats2 = pd.read_csv('data/stats2.csv')
stats = stats1.append(stats2)
teamstats = pd.read_csv('data/teamstats.csv')

C:\Users\tim\Anaconda3\envs\lol-predict\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


###### Data cleaning

In [4]:
df = pd.merge(participants, stats, how = 'left', on = ['id'], suffixes=('', '_y'))
df = pd.merge(df, matches, how = 'left', left_on = ['matchid'], right_on = ['id'], suffixes=('', '_y'))
print(df.shape)

(1834520, 71)


In [5]:
df = df[df["seasonid"] == 8]#we take in season 8 matches only
print(df.shape)

(1733658, 71)


In [6]:
def final_position(row):
    if row['role'] in ('DUO_SUPPORT', 'DUO_CARRY'):
        return row['role']
    else:
        return row['position']

df['adjustedPos'] = df.apply(final_position, axis = 1) 

df['team'] = df['player'].apply(lambda x: '1' if x <= 5 else '2')
df['team_role'] = df['team'] + ' - ' + df['adjustedPos']


In [6]:
# remove matchid with duplicate roles, e.g. 3 MID in same team, etc
remove_index = []
for i in ('1 - MID', '1 - TOP', '1 - DUO_SUPPORT', '1 - DUO_CARRY', '1 - JUNGLE', '2 - MID', '2 - TOP', '2 - DUO_SUPPORT', '2 - DUO_CARRY', '2 - JUNGLE'):
    df_remove = df[df['team_role'] == i].groupby('matchid').agg({'team_role':'count'})
    remove_index.extend(df_remove[df_remove['team_role']!=1].index.values)
    
# remove wrongly classified BOT, correct ones should be DUO_SUPPORT OR DUO_CARRY
remove_index.extend(df[df['adjustedPos'] == 'BOT']['matchid'].unique())
remove_index = list(set(remove_index))


In [7]:
#merge team stat into the main frame
teamstats['team'] = teamstats['teamid'].apply(lambda x: '1' if x == 100 else '2')
df = pd.merge(df, teamstats, how = 'left', on = ['matchid', 'team'], suffixes=('', '_y'))
df.head(10)

,id,matchid,player,championid,ss1,ss2,role,position,win,item1,...,firsttower,firstinhib,firstbaron,firstdragon,firstharry,towerkills,inhibkills_y,baronkills,dragonkills,harrykills
0,9,10,1,19,4,11,NONE,JUNGLE,0.0,3748.0,...,1,0,0,0,0,5,0,0,0,0
1,10,10,2,267,3,4,DUO_SUPPORT,BOT,0.0,2301.0,...,1,0,0,0,0,5,0,0,0,0
2,11,10,3,119,7,4,DUO_CARRY,BOT,0.0,1055.0,...,1,0,0,0,0,5,0,0,0,0
3,12,10,4,114,12,4,SOLO,TOP,0.0,1029.0,...,1,0,0,0,0,5,0,0,0,0
4,13,10,5,112,4,3,SOLO,MID,0.0,3020.0,...,1,0,0,0,0,5,0,0,0,0
5,14,10,6,72,11,4,NONE,JUNGLE,1.0,1400.0,...,0,1,1,1,1,10,3,1,3,1
6,15,10,7,3,4,12,SOLO,TOP,1.0,3025.0,...,0,1,1,1,1,10,3,1,3,1
7,16,10,8,103,14,4,SOLO,MID,1.0,3135.0,...,0,1,1,1,1,10,3,1,3,1
8,17,10,9,222,7,4,DUO_CARRY,BOT,1.0,3046.0,...,0,1,1,1,1,10,3,1,3,1
9,18,10,10,161,14,4,DUO_SUPPORT,BOT,1.0,1058.0,...,0,1,1,1,1,10,3,1,3,1


In [9]:
#final form
df = df[['id', 'matchid', 'player', 'championid', 'team_role', 'ss1', 'ss2', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'win', 'kills', 'deaths', 'assists', 'turretkills', 'inhibkills', 'totminionskilled', 'pinksbought', 'wardsplaced', 'duration', 'baronkills', 'harrykills', 'dragonkills', 'firsttower', 'firstinhib', 'firstblood',]]

In [10]:
df.head(20)

,id,matchid,player,championid,team_role,ss1,ss2,item1,item2,item3,...,totminionskilled,pinksbought,wardsplaced,duration,baronkills,harrykills,dragonkills,firsttower,firstinhib,firstblood
0,9,10,1,19,1 - JUNGLE,4,11,3748.0,2003.0,3111.0,...,42.0,0.0,10.0,1909,0,0,0,1,0,0.0
1,10,10,2,267,1 - DUO_SUPPORT,3,4,2301.0,3111.0,3190.0,...,17.0,1.0,17.0,1909,0,0,0,1,0,0.0
2,11,10,3,119,1 - DUO_CARRY,7,4,1055.0,3072.0,3006.0,...,205.0,1.0,13.0,1909,0,0,0,1,0,0.0
3,12,10,4,114,1 - TOP,12,4,1029.0,3078.0,3156.0,...,164.0,0.0,3.0,1909,0,0,0,1,0,0.0
4,13,10,5,112,1 - MID,4,3,3020.0,1058.0,3198.0,...,235.0,0.0,10.0,1909,0,0,0,1,0,0.0
5,14,10,6,72,2 - JUNGLE,11,4,1400.0,3111.0,3078.0,...,28.0,0.0,8.0,1909,1,1,3,0,1,0.0
6,15,10,7,3,2 - TOP,4,12,3025.0,3193.0,3068.0,...,187.0,1.0,7.0,1909,1,1,3,0,1,0.0
7,16,10,8,103,2 - MID,14,4,3135.0,3165.0,3089.0,...,183.0,0.0,8.0,1909,1,1,3,0,1,0.0
8,17,10,9,222,2 - DUO_CARRY,7,4,3046.0,3031.0,3006.0,...,191.0,1.0,6.0,1909,1,1,3,0,1,1.0
9,18,10,10,161,2 - DUO_SUPPORT,14,4,1058.0,2045.0,3092.0,...,72.0,5.0,25.0,1909,1,1,3,0,1,0.0


In [11]:
df.to_csv('cleanData.csv')